# Phase de Nettoyage des données

Les différentes données récupérées ont tout d'abord besoin d'être nettoyées pour les besoins du projet. Si les données fournies par l'EUDA sont déjà très propres et ne nécéssiteront qu'un léger travail de tri, voire éventuellement quelques merge utiles sur certaines tables, ce n'est pas le cas des données scrapées à partir de pages Wikipedia qui vont nous demander un travail de nettoyage plus approfondi.

## Nettoyage des données scrapées depuis Wikipedia

On commence par générer notre dataframe à partir du scraping des données de [l'article Wikipedia sur la légalisation du Cannabis par Pays](https://en.wikipedia.org/wiki/Legality_of_cannabis). Le code détaillé et commenté se trouve dans le fichier `scraping.py`, on se contente ici d'appeler nos fonctions.


In [16]:
from Data.scraping import scraping as s

url = "https://en.wikipedia.org/wiki/Legality_of_cannabis"

table = s.scrape_second_wikipedia_table(url)
data, header = s.table_to_list(table)
legal_pays = s.make_pandas(data, header)

legal_pays.head()

,Recreational,Medical,Notes
Afghanistan,Illegal,Illegal,Main article: Cannabis in Afghanistan Producti...
Albania,Illegal,Legal,Main article: Cannabis in Albania Prohibited b...
Algeria,Illegal,Illegal,Main article: Cannabis in Algeria
Andorra,Illegal,Illegal,Main article: Cannabis in Andorra
Angola,Illegal,Illegal,Main article: Cannabis in Angola


On remarque alors que notre Dataframe contient plusieurs éléments qui nécéssitent d'être nettoyés, modifiés, voire supprimés. Dans l'ordre on peut citer:

1. **Supprimer la dernier colonne concernant les "Notes"**, qui peuvent nous être utile à des fins d'interprétation, ou encore pour aider à la classification ou à l'interprétation plus tard, mais qui en l'état seront inutiles à la statistique descriptive ou à la construction de modèles plus tard.

2. **Eliminer de notre dataframe les pays non-européens**, puisque les données de l'EUDA ne s'intéresse qu'au cas européeen, et que c'est là le coeur de notre sujet.

3. Enfin, il nous faudra **regrouper les différentes états des variables sous un format clair** En l'état, on dispose pour chaque pays d'une description parfois unique et idividuelle de la législation dans le pays donné. L'idéal serait de ne garder qu'une variable binaire: Légal/Illégal. Cependant la réalité est plus complexe dans certains pays, et il nous faudra alors faire des choix, et les justifier, pour essayer de ne garder qu'un nombre limité d'état pour chaque variable.

### Suppression de la colonne des Notes

Cela n'est pas bien compliqué, on utilise simplement les outils de la librairie Pandas:

In [17]:
import pandas as pd

legal_pays_clean = legal_pays.drop(columns = "Notes")

legal_pays_clean.head()

,Recreational,Medical
Afghanistan,Illegal,Illegal
Albania,Illegal,Legal
Algeria,Illegal,Illegal
Andorra,Illegal,Illegal
Angola,Illegal,Illegal


### Tri pour ne garder que les pays Européens

Cette opération n'est elle aussi pas très compliquée, on va se servir d'une table de correspondance scrapée sur le [site national de Statistiques Candiennes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110) (ce afin d'avoir le nom des pays en anglais), qui nous sera d'ailleurs aussi utile plus tard afin de faire correspondre chaque pays de l'Union européenne avec son code ISO 3316.

In [18]:
from Data.scraping import scraping as s

url = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
table_corres = s.make_pandas(data)

table_corres.head()

,0,1
Belgium,BE,BEL
Bulgaria,BG,BGR
Croatia,HR,HRV
Cyprus,CY,CYP
Czechia,CZ,CZE


On doit néanmoins ajouter à la main certains pays non-membres de l'Union Européenne, mais présent dans les données que nous avons à disposition (souvent des pays candidats comme la Turquie):

In [19]:
import pandas as pd

table_corres.loc["Austria"] = ["AT","AUT"]
table_corres.loc["Türkiye"] = ["TR","TUR"]
table_corres.loc["Norway"] = ["NO","NOR"]

table_corres.tail()

,0,1
Spain,ES,ESP
Sweden,SE,SWE
Austria,AT,AUT
Türkiye,TR,TUR
Norway,NO,NOR


On effectue ensuite le tri des pays européens selon la manière suivante, en supprimant de la table principale toutes les lignes qui n'ont pas d'équivalent dans la table de correspondance.

In [20]:
# On renomme certains pays pour correspondre aux données de l'EUDA 
legal_pays_clean.rename(index={"Turkey":"Türkiye"}, inplace=True)
legal_pays_clean.rename(index={"Czech Republic":"Czechia"}, inplace=True)

for index in legal_pays_clean.index:
    if index in table_corres.index:
        pass
    else:
        legal_pays_clean = legal_pays_clean.drop(index = index)

legal_pays_clean.head()


,Recreational,Medical
Austria,Possession for personal use decriminalized as ...,"Dronabinol, Sativex, Nabilone[25]"
Belgium,Decriminalized up to 3 g (1⁄10 oz) or cultivat...,Sativex[31]
Bulgaria,Illegal,Illegal
Croatia,Decriminalized[71],Legal[72]
Cyprus,Illegal,Legal[73]


### Recodage de chaque élément dans une variable ternaire

Il nous faut désormais faire un travail de recodage des informations contenues dans le tableau, afin de ne garder au final qu'une variable avec trois états possibles:

- `Legal` si la consommation, la possession et la vente sont légales et encadrées dans le pays
- `Illegal` si la consommation, la possession et la vente sont illégale et passibles de poursuites judiciaires
- `Decriminalized` si la consommation et la possession sont décriminalisées, mais la vente est illégale. On ne fait pas ici de différence entre la pénalisation ou non du cannabis (la seule différence étant que dans le cas ou ce dernier est pénalisé, la consommation et la possession sont toujours passibles d'une amende, mais pas de poursuites légales)

L'information est souvent contenue en clair dans notre tableau, mais perdue au milieu de précisions et de nuances propres à chaque pays. Par souci de simplicité, on appliquera les règles suivantes afin de recoder efficacement le tableau:

- **On ignorera toute limite de poids ou de nombres de plants**: Par exemple dans le cas de la République Tchèque, la possession de cannabis est décriminalisée jusqu'à 10g, et illégale au delà. On la considère donc simplement comme décriminalisée.

- **Dans le domaine médical, l'autorisation du Sativex vaut pour légalisation, celle du CBD non**: En effet, le Sativex contient du THC tout comme certains autres médicaments dérivés de cannabis, ce qui n'est pas le cas pour le CBD (ou alors en quantité très faible). En outre, au vu du fait que le CBD soit légal dans la plupart des pays, considérer son autorisation comme valant légalisation aurait pour conséquence d'appauvrir largement nos données.

- **On se tiendra à la stricte législation théorique, sans prendre en compte pour l'instant son application pratique**, cela aussi par souci de simplicité car la situation de chaque pays est unique et complexe. Par exemple, en Finlande, la possession et la consommation de cannabis est illégale *stricto sensu* et peut valoir une peine de prison. En réalité, les consommateurs s'en tirent souvent avec une simple amende; ce qui serait presque équivalent *en pratique* à une décriminalisation. Le fait est qu'il nous est impossible de trancher pour chaque pays à quel moment s'arrête la théorie et commence la pratique. Nous n'avons en effet ni la compétence sur le sujet, ni l'autorité nécessaire pour effectuer ce genre d'arbitrage, ce qui nous amène à ne prendre en compte que la législation en vigueur, et non son application réelle.
En revanche, il pourra nous être utile de tenir compte de ces différences d'application de la loi lors de l'analyse que nous ferons de nos données, mais pas dans la construction de celles-ci.

En prenant cela en compte, nous pouvons recoder le tableau de la manière suivante:

In [21]:
# Recodage des information selon les règles fixées plus haut

for index in legal_pays_clean.index:

    content = legal_pays_clean.loc[index].values.flatten().tolist()
    
    for i in range(len(content)):
        if "Legal" in content[i] or "Sativex" in content[i]:
            content[i] = "Legal"
        elif "Illegal" in content[i] or "CBD" in content[i]:
            content[i] = "Illegal"
        elif "Decriminalized" in content[i] or "decriminalized" in content[i]:
            content[i] = "Decriminalized"

    legal_pays_clean.loc[index] = content

print(legal_pays_clean)

               Recreational                                            Medical
Austria      Decriminalized                                              Legal
Belgium      Decriminalized                                              Legal
Bulgaria            Illegal                                            Illegal
Croatia      Decriminalized                                              Legal
Cyprus              Illegal                                              Legal
Czechia      Decriminalized                                              Legal
Denmark             Illegal                                              Legal
Estonia      Decriminalized                          With a special permit[88]
Finland             Illegal                                              Legal
France              Illegal  A two-year trial program involving 3,000 patie...
Germany               Legal                                              Legal
Greece              Illegal                         

Puisqu'il reste après filtrage encore quelques valeurs qui n'ont pas de définition claire, nous n'avons d'autre choix que de les recoder à la main, en justifiant bien évidemment nos choix à chaque étape:

- En Estonie, l'usage du cannabis pour la médecine est autorisé avec "un permis spécial". Dans les faits, une recherche plus détaillée sur ce permis nous apprend qu'il n'est en réalité jamais délivré et qu'un seul patient a ce jour a été autorisé à recevoir du Sativex. Cependant, pour être cohérent avec notre règle fixée plus haut de ne prendre en compte que la législation théorique en vigueur, on choisit de considérer l'usage comme Légal, puisqu'il est théoriquement possible.

- En France, l'usage cannabis thérapeutique est aujourd'hui sujet d'expérimentation dans certains cas précis et très encadrés, mais reste en théorie illégal. On choisit donc de le coder comme étant illégal puisque les expérimentations encadrées n'ouvrent pas à une possibilité d'utilisation pour la population générale.

- En Roumanie, l'usage thérapeutique de dérivés de cannabis avec une teneur en THC inférieure à 0,2% est autorisée. Etant donnée que la plupart des pays (dont la France) autorisent le CBD à condition qu'il ne contienne pas plus de 0,3% de THC, on choisit de considérer l'usage thérapeutique en Roumanie comme illégal, puisqu'on s'est fixé comme règle plus haut que la légalisation du CBD ne valait pas celle du cannabis.

In [22]:
# Recodage à la main des informations plus ambigues

legal_pays_clean.loc['France','Medical'] = 'Illegal'
legal_pays_clean.loc['Estonia','Medical'] = 'Legal'
legal_pays_clean.loc['Romania','Medical'] = 'Illegal'

legal_pays_clean.head()

,Recreational,Medical
Austria,Decriminalized,Legal
Belgium,Decriminalized,Legal
Bulgaria,Illegal,Illegal
Croatia,Decriminalized,Legal
Cyprus,Illegal,Legal


On obtient donc finalement un tableau complètement nettoyé et exploitable pour nos analyses futures !